In [1]:
from sklearn import datasets

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,GlobalAveragePooling2D,Input

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
device=torch.device('cuda')

In [2]:
print(tf.__version__)
print(torch.__version__)

2.3.1
1.7.1


Generate data pipeline of building a model

Model:

    Forward propagation:
        Layer
        activation function
    Loss function
    Optimizer:
        Batch gradient desent
    Metric
    Back propagation
    Evaluation
    Prediction
Save model


Preparing data

In [3]:
digits = datasets.load_digits()

In [4]:
'''For Tensorflow, channel last for input'''
features_train=digits.images[:-5][:,:,:,np.newaxis]
target_train=digits.target[:-5]

features_test=digits.images[-5:][:,:,:,np.newaxis]
target_test=digits.target[-5:]

'''For Pytorch, channel first for input'''
features_torch_train=features_train.reshape(features_train.shape[0],-1,8,8)
features_torch_test=features_test.reshape(features_test.shape[0],-1,8,8)

print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_torch_train.shape)

(1792, 8, 8, 1)
(1792,)
(5, 8, 8, 1)
(5,)
(1792, 1, 8, 8)


In [5]:
#keras API
'''
tf.keras.preprocessing.image_dataset_from_directory
tf.keras.preprocessing.text_dataset_from_directory
'''
#Tensorflow
tf_dataset_train = tf.data.Dataset.from_tensor_slices((features_train, target_train))
tf_dataset_test = tf.data.Dataset.from_tensor_slices((features_test, target_test))
#Pytorch (Pandas -> Numpy -> Tensor)
'''target: dtype=torch.long for nn.CrossEntropyLoss()'''
torch_dataset_train = torch.utils.data.TensorDataset(
    torch.tensor(features_torch_train,device=device,dtype=torch.float32),
    torch.tensor(target_train,device=device,dtype=torch.long))

torch_dataset_test = torch.utils.data.TensorDataset(
    torch.tensor(features_torch_test,device=device,dtype=torch.float32),
    torch.tensor(target_test,device=device,dtype=torch.long))

In [6]:
#Visiualizing data in tensorflow and pytorch

In [7]:
print('Tensorflow')
print(f"feature shape: {next(iter(tf_dataset_train))[0].shape},target shape: {next(iter(tf_dataset_train))[1].shape}")
print()
print('Pytorch')
print(f"feature shape: {next(iter(torch_dataset_train))[0].shape},target shape: {next(iter(torch_dataset_train))[1].shape}")

Tensorflow
feature shape: (8, 8, 1),target shape: ()

Pytorch
feature shape: torch.Size([1, 8, 8]),target shape: torch.Size([])


Batching

In [8]:
#Tensorflow
tf_loader_train=tf_dataset_train.shuffle(len(tf_dataset_train)).batch(32)
'''No shuffle for test data'''
tf_loader_test=tf_dataset_test.batch(32)
#Pytorch
torch_loader_train=torch.utils.data.DataLoader(torch_dataset_train,batch_size=32,shuffle=True)
'''No shuffle for test data'''
torch_loader_test=torch.utils.data.DataLoader(torch_dataset_test,shuffle=False)

In [9]:
#Tensorflow
print('Tensorflow')
print(f"feature shape: {next(iter(tf_loader_train))[0].shape}, target shape: {next(iter(tf_loader_train))[1].shape}")
print()
print(f"feature shape: {next(iter(torch_loader_train))[0].shape}, target shape: {next(iter(torch_loader_train))[1].shape}")

Tensorflow
feature shape: (32, 8, 8, 1), target shape: (32,)

feature shape: torch.Size([32, 1, 8, 8]), target shape: torch.Size([32])


In [10]:
#Transformation

Train model:
    Forward propagation

In [11]:
#Keras API
input_shape = (8,8,1)
classes=10

def Keras_API(input_shape,classes):
    X_input = keras.layers.Input(input_shape,name='Input')
    X = Conv2D(filters=32, kernel_size=(3, 3), activation="relu",name="Conv1")(X_input)
    X = Conv2D(filters=32, kernel_size=(3, 3), activation="relu",name="Conv2")(X)
    X = MaxPooling2D(pool_size=(2, 2),name="MaxPool")(X)
    X = GlobalAveragePooling2D(name="GlobalPool")(X)
    outputs = Dense(classes, activation="softmax",name="Connected")(X)
    model = keras.models.Model(inputs = X_input, outputs = outputs, name='Keras_API')
    return model

model=Keras_API(input_shape,classes)
print(model.summary())

'''Initiate 
optimizer, loss and metric in model.compile
batch_size, epochs in model.fit'''
loss_fn=keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='Adam',loss=loss_fn,metrics='sparse_categorical_accuracy')
model.fit(features_train, target_train,batch_size=32, epochs=10,verbose=1)


Model: "Keras_API"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 8, 8, 1)]         0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 6, 6, 32)          320       
_________________________________________________________________
Conv2 (Conv2D)               (None, 4, 4, 32)          9248      
_________________________________________________________________
MaxPool (MaxPooling2D)       (None, 2, 2, 32)          0         
_________________________________________________________________
GlobalPool (GlobalAveragePoo (None, 32)                0         
_________________________________________________________________
Connected (Dense)            (None, 10)                330       
Total params: 9,898
Trainable params: 9,898
Non-trainable params: 0
_______________________________________________________

In [12]:
model.evaluate(features_test,target_test)

pred_test=model.predict(features_test)
'''
Pred_test is an array with the probability of 10 classes
Use np.argmax to extract the index of highest porbability
'''
print(np.argmax(pred_test,axis=1))
print(target_test)

1/1 [==============================] - 0s 0s/step - loss: 0.0880 - sparse_categorical_accuracy: 1.0000
[9 0 8 9 8]
[9 0 8 9 8]


In [13]:
#Tensorflow
'''
You can self defined layer and model via
tf.keras.layers.Layer
tf.keras.Model
'''
'''
class self_define_layer(tf.keras.layers.Layer):
    def __init__(self,variable):
        super(self,self_define_layer).__init__()
    def call(self,):
        return
'''
class TF_model(keras.Model):
    def __init__(self,filters=32,kernel=(3,3),pool_size=(2,2),classes=10):
        super(TF_model,self).__init__()
        self.filters=filters
        self.kernel=kernel
        self.pool_size=pool_size
        self.classes=classes
        
        self.conv2d_1=Conv2D(filters=self.filters, kernel_size=self.kernel, activation="relu")
        self.conv2d_2=Conv2D(filters=self.filters, kernel_size=self.kernel, activation="relu")
        self.maxpool=MaxPooling2D(pool_size=self.pool_size)
        self.globalpool=GlobalAveragePooling2D()
        self.dense=Dense(self.classes, activation="softmax")
        
    def call(self,inputs):
        x=self.conv2d_1(inputs)
        x=self.conv2d_2(x)
        x=self.maxpool(x)
        x=self.globalpool(x)
        output=self.dense(x)
        return output
TF=TF_model()

In [14]:
'''Clean model parameters for previous training'''
tf.keras.backend.clear_session()

epochs=10
'''Define Optimizer and Metric'''
loss_fn=keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()

for epoch in range(epochs):
    train_acc_metric.reset_states()
    for step, (x_batch_train, y_batch_train) in enumerate(tf_loader_train):
        
        with tf.GradientTape() as tape:
            pred = TF(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, pred)

        gradients = tape.gradient(loss_value, TF.trainable_weights)
        optimizer.apply_gradients(zip(gradients, TF.trainable_weights))
        # Update training metric.
        train_acc_metric.update_state(y_batch_train,pred)
    train_acc = train_acc_metric.result()
    print(f"Epoch:{epoch+1}, loss: {loss_value.numpy()}, Accuracy: {train_acc.numpy()}", )


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch:1, loss: 1.0790282487869263, Accuracy: 0.5128348469734192
Epoch:2, loss: 0.39115095138549805, Accuracy: 0.875
Epoch:3, loss: 0.3408812880516052, Accuracy: 0.9285714030265808
Epoch:4, loss: 0.14485768973827362, Accuracy: 0.9425223469734192
Epoch:5, loss: 0.17826536297798157, Accuracy: 0.9620535969734192
Epoch:6, loss: 0.10369610786437988, Accuracy: 0.9659598469734192
Epoch:7, loss: 0.11171208322048187, Accuracy: 0.9659598469734192
Epoch:8, loss: 0.14070822298526764, Accuracy: 0.9743303656578064
Epo

In [15]:
TF.compile(loss=loss_fn, metrics=train_acc_metric)
TF.evaluate(tf_loader_test)
pred_test=TF.predict(tf_loader_test)
print(np.argmax(pred_test,axis=1))
print(list(iter(tf_loader_test))[0][1])

1/1 [==============================] - 0s 1ms/step - loss: 0.0194 - sparse_categorical_accuracy: 0.9783
[9 0 8 9 8]
tf.Tensor([9 0 8 9 8], shape=(5,), dtype=int32)


In [16]:
#Pytorch
input_shape=(1,8,8)
class Pytorch_model(nn.Module):
    def __init__(self,filters=32,kernel=(3,3),pool_size=(2,2),classes=10):
        super(Pytorch_model, self).__init__()
        self.filters=filters
        self.kernel=kernel
        self.pool_size=pool_size
        self.classes=classes
        ''' For Conv2d: (input channel, output channel(number of filters), kernel size)'''
        self.Conv1=nn.Conv2d(1, filters ,kernel_size = kernel)
        self.Conv2=nn.Conv2d(32, filters ,kernel_size = kernel)
        self.MaxPool=nn.MaxPool2d(self.pool_size)
        '''Average Pooling:Dimension of the last feature map'''
        self.AvgPool=nn.AvgPool2d(2)
        self.Dense=nn.Linear(filters*1*1,classes)
        
    def forward(self,inputs):
        X = F.relu(self.Conv1(inputs))
        X = F.relu(self.Conv2(X))
        X = self.MaxPool(X)
        X = self.AvgPool(X)
        '''Flatten tensor'''
        X = X.view(-1, 32 * 1 * 1)
        X = self.Dense(X)
        return X
    
pytorch_model=Pytorch_model().cuda()
print(summary(pytorch_model, input_shape))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 6, 6]             320
            Conv2d-2             [-1, 32, 4, 4]           9,248
         MaxPool2d-3             [-1, 32, 2, 2]               0
         AvgPool2d-4             [-1, 32, 1, 1]               0
            Linear-5                   [-1, 10]             330
Total params: 9,898
Trainable params: 9,898
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.04
Estimated Total Size (MB): 0.05
----------------------------------------------------------------
None


In [17]:
import torch.optim as optim
'''Seems no metric for pytorch, but found metric function in pytorch lightning'''

'''for CrossEntropyLoss(), the shape of target should be 1-D tensor, ie,torch.Size([batch size]'''
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.0001,betas=(0.9, 0.999))

epochs=10
running_loss = 0.0

for epoch in range(epochs):
    for step, (x_batch_train, y_batch_train) in enumerate(torch_loader_train):
        
        optimizer.zero_grad()
        outputs = pytorch_model(x_batch_train)
        loss = criterion(outputs, y_batch_train)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print(f"Epoch:{epoch+1}, loss: {loss}", )

Epoch:1, loss: 2.192021369934082
Epoch:2, loss: 2.0370450019836426
Epoch:3, loss: 1.9012563228607178
Epoch:4, loss: 1.6298552751541138
Epoch:5, loss: 1.3796319961547852
Epoch:6, loss: 1.3091481924057007
Epoch:7, loss: 0.9962636828422546
Epoch:8, loss: 0.8420236110687256
Epoch:9, loss: 0.6224917769432068
Epoch:10, loss: 0.6451207995414734


In [18]:
pred_pytorch=[]
targets=[]
with torch.no_grad():
        for data,target in torch_loader_test:
            pred=pytorch_model(data)
            pred_pytorch.append(torch.argmax(pred).item())
            targets.append(target.item())
print(pred_pytorch)
print(targets)

[9, 0, 8, 9, 8]
[9, 0, 8, 9, 8]
